## Import Libraries

In [1]:
import sys
from glob import glob
from osgeo import ogr, gdal
from osgeo import gdalconst
import subprocess

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap, BoundaryNorm
import numpy as np

import seaborn as sns

import json

import geopandas as gpd
import pycrs
import fiona
from fiona.crs import from_epsg
from shapely.geometry import box
from shapely.geometry import Point
import shapely.geometry as geoms
from shapely.geometry import Polygon

import rasterio as rio
from rasterio.plot import show
import rasterio.warp
import rasterio.shutil
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.plot import plotting_extent
from rasterio.plot import show_hist
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio import Affine, MemoryFile
from rasterio.enums import Resampling
from rasterio import plot

import rasterstats as rs
import georasters as gr
from rastertodataframe import raster_to_dataframe

import earthpy.spatial as es
import earthpy.plot as ep
import earthpy as et

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate

%matplotlib inline
#setting consistent plotting settings
sns.set_style("white")
sns.set(font_scale=1.5)
pd.set_option('display.max_rows',100)

## Libraries versions

In [2]:
print(f"Python {sys.version}")
print(f"geopandas {gpd.__version__}")
print(f"osgeo {gdal.__version__}")
print(f"rasterio {rasterio.__version__}")

Python 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]
geopandas 0.9.0
osgeo 3.3.2
rasterio 1.2.9


## Extract pixel values from the tif

In [3]:
def extract_values(arr):
    df = pd.DataFrame(arr.reshape(-1,4), columns={'B1','B2','B3','B4'})
    drop_index = df.query("B1 == 0 and B2 == 0 and B3 == 0 and B4 == 0").index
    df.drop(df.index[[drop_index]], inplace = True)
    df.reset_index(drop = True,inplace = True)
    #df = pd.to_numeric(df, downcast='float')
    return df


## Vegetation indices functions

Generate veg indices

- B1 = RED
- B2 = GREEN
- B3 = BLUE
- B4 = NIR


In [4]:
def ndvi(b4,b1):
    b4 = b4.astype(int);b1 = b1.astype(int)
    return (b4 - b1) / (b4 + b1)

def gndvi(b4,b2):
    b4 = b4.astype(int);b2 = b2.astype(int)
    return (b4 - b2) / (b4 + b2)

def gci(b4,b2):
    b4 = b4.astype(int);b2 = b2.astype(int)
    return b4 / (b2 -1)

def sipi(b4,b3,b1):
    b4 = b4.astype(int);b3 = b3.astype(int);b1 = b1.astype(int)
    return (b4 - b3) / (b4 - b1)

def arvi(b4,b1,b3):
    b4 = b4.astype(int);b3 = b3.astype(int);b1 = b1.astype(int)
    return (b4 - 2*b1 + b3) / (b4 + 2*b1 + b3)

def savi(b4,b1,l=1):
    b4 = b4.astype(int);b1 = b1.astype(int)
    return ((b4 - b1) / (b4 + b1+1))*(1+l)

def evi(b4,b3,b1,c1 = 0.5,c2 = 0.5,l = 1):
    b4 = b4.astype(int);b3 = b3.astype(int);b1 = b1.astype(int)
    return ((b4 - b1) / (b4 + (c1*b1) - (c2*b3) + l))*2.5


In [5]:
def veg_indices(df,date):
    df['ndvi'] = ndvi(df.B4,df.B1)
    df['gndvi'] = gndvi(df.B4,df.B2)
    df['gci'] = gci(df.B4,df.B2)
   # df['sipi'] = sipi(df.B4,df.B3,df.B1)
    df['arvi'] = arvi(df.B4,df.B3,df.B1)
    df['savi'] = savi(df.B4,df.B2)
    df['evi'] = evi(df.B4,df.B3,df.B1)
    df.columns = np.array(df.columns) + "_" + str(date)
    return df
#veg_indices(res,"20150508")


## Extracting each plots pixel values from the tiffs

- Clip the plot to the tiff
- Mask it out
- Extract band values
- Append vegetation indices
- Sample aggregated sections of the plot
- Save the dataframe as CSV


In [89]:
shapein.Grown_crop.value_counts()

1    1369
0     320
3     156
2      87
4      70
Name: Grown_crop, dtype: int64

In [108]:
def extract_plot_info(poly,tiff,date = "20150508"):
    try:
        out_img, out_transform = mask(dataset=tiff, shapes= [poly['geometry']], crop=True)
    except:
        return []
    else:
        tiff_meta.update({"driver": "GTiff",
                 "height": out_img.shape[1],
                 "width": out_img.shape[2],
                 "transform": out_transform,
                 "crs": crs}
                         )
        df = veg_indices(extract_values(out_img),date)
    
        return df
        
      

## Import and Export extracted plot information

In [110]:
# shapein is the shapefile GT
shapein = "E:\JANET\DG_pansharpen_images\kofa_redu\kofa_redu.shp"
shapein= gpd.GeoDataFrame.from_file(shapein)

#The labels to append
labels = list(shapein.Grown_crop)
# Load the tiff
tif = "../worldview3_tile_01_20150508.tif"
tif=rio.open(tif)
# Copy the metadata
tiff_meta = tif.meta.copy()
crs = tif.crs
#print(tiff_meta)
# convert  shapefile to  geojson format to  get the geometry features
plots = json.loads(shapein.to_json())['features']

#Example  
dates = ["20150508"]
for j in range(len(dates)):
    for i in range(len(plots)):
        df = extract_plot_info(poly = plots[i],tiff = tif,date = dates[j])
        if(len(df) > 0):
            df.fillna(0,inplace = True)
            df.insert(1,'Grown_crop', labels[i])

            if df.Grown_crop.unique() == 0:
                drop_last_rows = df.shape[0] - df.shape[0]//100 * 100
                df.drop(df.tail(drop_last_rows).index,inplace = True)
                df['group'] = np.repeat(["a_"+str(x) for x in range(100)], df.shape[0]//100, axis=0)
                df= df.groupby("group", as_index = False,sort = False).mean()
            elif df.Grown_crop.unique() == 1:
                drop_last_rows = df.shape[0] - df.shape[0]//24 * 24
                df.drop(df.tail(drop_last_rows).index,inplace = True)
                df['group'] = np.repeat(["a_"+str(x) for x in range(24)], df.shape[0]//24, axis=0)
                df= df.groupby("group", as_index = False,sort = False).mean()
            elif df.Grown_crop.unique() == 2:
                drop_last_rows = df.shape[0] - df.shape[0]//360 * 360
                df.drop(df.tail(drop_last_rows).index,inplace = True)
                df['group'] = np.repeat(["a_"+str(x) for x in range(360)], df.shape[0]//360, axis=0)
                df= df.groupby("group", as_index = False,sort = False).mean()
            elif df.Grown_crop.unique() == 3:
                drop_last_rows = df.shape[0] - df.shape[0]//200 * 200
                df.drop(df.tail(drop_last_rows).index,inplace = True)
                df['group'] = np.repeat(["a_"+str(x) for x in range(200)], df.shape[0]//200, axis=0)
                df= df.groupby("group", as_index = False,sort = False).mean()
            else:
                drop_last_rows = df.shape[0] - df.shape[0]//480 * 480
                df.drop(df.tail(drop_last_rows).index,inplace = True)
                df['group'] = np.repeat(["a_"+str(x) for x in range(480)], df.shape[0]//480, axis=0)
                df= df.groupby("group", as_index = False,sort = False).mean()
           # res =pd.DataFrame(res)
            #res.insert(1,'Grown_crop', labels[i])
            df.to_csv("E:/JANET/DG_pansharpen_images/May08_data/"+"plot_"+str(i)+'_'+df.columns[1][3:]+".csv", index = False)
        else:
            pass
        

In [12]:
df.columns[1][3:]

'20150508'

## Merging the dataframes per plot

### some Plots skipped in date 20150925

We average from the previous and after date

Plots: 280,1160,1637,1781,1912 for September 

In [123]:
plot_280_20150912 = pd.read_csv(r"E:\JANET\DG_pansharpen_images\Sep12_data\plot_280_20150912.csv")

In [124]:
plot_280_20151121 = pd.read_csv(r"E:\JANET\DG_pansharpen_images\Nov21_data\plot_280_20151121.csv")

In [125]:
plot_280_20150925 = pd.DataFrame()
plot_280_20150925['group']=plot_280_20151121.group  
plot_280_20150925['Grown_crop'] = plot_280_20151121.Grown_crop
plot_280_20150925['B4_20150925'] = (plot_280_20150912.B4_20150912 + plot_280_20151121.B4_20151121)/2
plot_280_20150925['B2_20150925'] = (plot_280_20150912.B2_20150912 + plot_280_20151121.B2_20151121)/2
plot_280_20150925['B1_20150925'] = (plot_280_20150912.B1_20150912 + plot_280_20151121.B1_20151121)/2
plot_280_20150925['B3_20150925'] = (plot_280_20150912.B3_20150912 + plot_280_20151121.B3_20151121)/2
plot_280_20150925['ndvi_20150925'] = (plot_280_20150912.ndvi_20150912 + plot_280_20151121.ndvi_20151121)/2
plot_280_20150925['gndvi_20150925'] = (plot_280_20150912.gndvi_20150912 + plot_280_20151121.gndvi_20151121)/2
plot_280_20150925['gci_20150925'] = (plot_280_20150912.gci_20150912 + plot_280_20151121.gci_20151121)/2
plot_280_20150925['arvi_20150925'] = (plot_280_20150912.arvi_20150912 + plot_280_20151121.arvi_20151121)/2
plot_280_20150925['savi_20150925'] = (plot_280_20150912.savi_20150912 + plot_280_20151121.savi_20151121)/2
plot_280_20150925['evi_20150925'] = (plot_280_20150912.evi_20150912 + plot_280_20151121.evi_20151121)/2
 
    
plot_280_20150925.to_csv("E:/JANET/DG_pansharpen_images/Sept25_data/plot_280_20150925.csv", index = False)

## Merge plots data to a CSV

In [2]:
import datetime
#the obeserved dates in the dataset
dates_raw = [
datetime.datetime(2015, 5, 8, 0, 0),
datetime.datetime(2015, 5, 22, 0, 0),
datetime.datetime(2015, 6, 3, 0, 0),
datetime.datetime(2015, 6, 29, 0, 0),
datetime.datetime(2015, 7, 24, 0, 0),
datetime.datetime(2015, 9, 12, 0, 0),
datetime.datetime(2015, 9, 25, 0, 0),
datetime.datetime(2015, 11, 21, 0, 0)
]

dates = []

for i in range(8):
    dt = "".join(str(dates_raw[i].date()).split("-"))
    dates.append(dt)

### For the first date include the plot ID.

In [3]:
## This reads all plot files for each date
data_20150508 = []
for t in range(2002):
    file_name = pd.read_csv(f"E:\JANET\DG_pansharpen_images\Plots_data\Individual_plots_data\plot_{t}_20150508.csv")
    # Include the plot id
    file_name['ID'] = t
    data_20150508.append(file_name)
        #return data
data_20150508 = pd.concat(data_20150508  , ignore_index=False, sort=True)
data_20150508.to_csv("E:\JANET\DG_pansharpen_images\Plots_data\Combined_plots_data\Data_20150508"+".csv")

In [ ]:
## This reads all plot files for the remaining dates and exclude the plot id
data_20150522 = []
for t in range(2002):
    file_name = pd.read_csv(f"E:\JANET\DG_pansharpen_images\Plots_data\Individual_plots_data\plot_{t}_20150522.csv")
    file_name['ID'] = t
    data_20150522.append(file_name)
        #return data
data_20150522 = pd.concat(data_20150522  , ignore_index=False, sort=True)
data_20150522.to_csv("E:\JANET\DG_pansharpen_images\Plots_data\Combined_plots_data\Data_20150522"+".csv")

In [8]:
data_20150508.shape

(160976, 13)

In [9]:
### Combine all dataframes for all months.
Combined_data= []
for date in dates:
    file_name = pd.read_csv(f"E:\JANET\DG_pansharpen_images\Plots_data\Combined_plots_data\Data_{date}.csv") 
    Combined_data.append(file_name)
        #return data
Combined_data = pd.concat(Combined_data, ignore_index=False, sort=True,axis =1)
Combined_data.to_csv("E:\JANET\DG_pansharpen_images\PLots_data\Combined_plots_data\Combined_data"+".csv")